<div class="alert" style="background-color:#006400; color:white; padding:0px 10px; border-radius:5px;"><h1 style='margin:15px 15px; color:#FFFFFF; font-size:32px'>Recurrent Neuron Network Plus</h1></div>

The work is under the **"Master Thesis"** by **Chau Tran** with the supervision from **Prof. Roland Olsson**.

v1_5:
* 04/01/2021: Testing RNN plus in real-world problems (datasets)

In [ ]:
import tensorflow as tf
import tensorboard
import pandas as pd
import numpy as np
import sys, os, math, time, datetime, re
from sklearn.model_selection import train_test_split

print("tf: ", tf.__version__)
print("tb: ", tensorboard.__version__)
print(os.getcwd())

tf.get_logger().setLevel('ERROR')
tf.autograph.set_verbosity(0)
# tf.config.set_visible_devices([], 'GPU')
os.environ['CUDA_VISIBLE_DEVICES'] = '-1'
os.environ['TF_ENABLE_ONEDNN_OPTS'] = '1'
tf.keras.backend.set_floatx('float64')

ISMOORE_DATASETS = False

# Debugging with Tensorboard
snapshot = datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
logdir="../logs/fit/rnn_v1_1/" + snapshot

path = "../../Datasets/2_addingproblem" #"../../Datasets/0_180_small_datasets/Version9.128timesteps"
fileslist = [f for f in sorted(os.listdir(path)) if os.path.isfile(os.path.join(path, f))]

with open("./params/params_addingproblem.txt") as f:
    hyperparams = dict([re.sub('['+' ,\n'+']','',x.replace(' .', '')).split('=') for x in f][1:-1])
hyperparams = dict([k, float(v)] for k, v in hyperparams.items())
hyperparams['testSize'] = 0.500
print(hyperparams)

def seperateValues(data, noInput, noOutput, isMoore=True):
    x_data, y_data = None, None
    for i in range(data.shape[0]):
        if isMoore:
            x_data_i = data[i].reshape(-1, noInput+noOutput)
            x_data_i, y_data_i = x_data_i[:, 0:noInput], x_data_i[-1, noInput:]
        else:
            x_data_i = data[i][:-1].reshape(-1, noInput)
            y_data_i = data[i][-1].reshape(-1, noOutput)
        x_data = x_data_i[np.newaxis,:,:] if x_data is None else np.append(x_data, x_data_i[np.newaxis,:,:], axis=0)
        y_data = y_data_i.reshape(1, -1) if y_data is None else np.append(y_data, y_data_i.reshape(1, -1), axis=0)
    return x_data, y_data

def fromBit( b ) :
    return -0.9 if b == 0.0 else 0.9

class CustomMetricError(tf.keras.metrics.MeanMetricWrapper):
    def __init__(self, name='custom_metric_error', dtype=None, threshold=0.5):
        super(CustomMetricError, self).__init__(
            customMetricfn_tensor, name, dtype=dtype, threshold=threshold)

def customMetricfn_tensor(true, pred, threshold=0.5):
    true = tf.convert_to_tensor(true)
    pred = tf.convert_to_tensor(pred)
    threshold = tf.cast(threshold, pred.dtype)
    pred = tf.cast(pred >= threshold, pred.dtype)
    true = tf.cast(true >= threshold, true.dtype)
    return keras.backend.mean(tf.equal(true, pred), axis=-1)

def customMetricfn(y_true, y_pred):
    count, numCorrect = 0, 0
    for i in range( y_true.shape[0] ) :
        for j in range( y_pred.shape[ 1 ] ) :
            count += 1
            if isCorrect( y_true[ i, j ], y_pred[ i, j ] ) :
                numCorrect += 1
    return (numCorrect/count)

def isCorrect( target, actual ) :
    y1 = False if target < 0.0 else True
    y2 = False if actual < 0.0 else True
    return y1 == y2 

class customLRSchedule(tf.keras.optimizers.schedules.LearningRateSchedule):
    def __init__(self, initialLearningRate, learningRateDecay, decayDurationFactor, numTrainingSteps, glorotScaleFactor=0.1, orthogonalScaleFactor=0.1, name=None):
        self.initialLearningRate = initialLearningRate
        self.learningRateDecay = learningRateDecay
        self.decayDurationFactor = decayDurationFactor
        self.glorotScaleFactor = glorotScaleFactor
        self.orthogonalScaleFactor = orthogonalScaleFactor
        self.numTrainingSteps = numTrainingSteps
        self.name = name
        self.T = tf.constant(self.decayDurationFactor * self.numTrainingSteps, dtype=tf.float32, name="T")
    
    def __call__(self, step):
        self.step = tf.cast(step, tf.float32)
        self.lr = tf.cond(self.step > self.T, 
                           lambda: tf.constant(self.learningRateDecay * self.initialLearningRate, dtype=tf.float32),
                           lambda: self.initialLearningRate * (1.0 - (1.0 - self.learningRateDecay) * self.step / self.T)
                          )
        return self.lr
    
class RNN_plus_v1_cell(tf.keras.layers.Layer):
    def __init__(self, units, kernel_initializer='glorot_uniform', recurrent_initializer='orthogonal', bias_initializer='zeros', dropout=0., recurrent_dropout=0., use_bias=True, **kwargs):
        if units < 0:
            raise ValueError(f'Received an invalid value for argument `units`, '
                                f'expected a positive integer, got {units}.')
        # By default use cached variable under v2 mode, see b/143699808.
        if tf.compat.v1.executing_eagerly_outside_functions():
            self._enable_caching_device = kwargs.pop('enable_caching_device', True)
        else:
            self._enable_caching_device = kwargs.pop('enable_caching_device', False)
        super(RNN_plus_v1_cell, self).__init__(**kwargs)
        self.units = units
        self.state_size = self.units
        self.output_size = self.units
        self.kernel_initializer = tf.keras.initializers.get(kernel_initializer)
        self.recurrent_initializer = tf.keras.initializers.get(recurrent_initializer)
        self.bias_initializer = tf.keras.initializers.get(bias_initializer)
        self.dropout = min(1., max(0., dropout))
        self.recurrent_dropout = min(1., max(0., recurrent_dropout))
        self.use_bias = True
    
    def build(self, input_shape):
        self.kernel = self.add_weight(shape=(input_shape[-1], self.units), name='w_i', initializer=self.kernel_initializer, regularizer=None, constraint=None)
        self.recurrent_kernel = self.add_weight(shape=(self.units, self.units), name='w_o', initializer=self.recurrent_initializer, regularizer=None, constraint=None)
        self.bias = self.add_weight( shape=(self.units,), name='b', initializer=self.bias_initializer, regularizer=None, constraint=None) if self.use_bias else None
        self.built = True
        
    def call(self, inputs, states, training=None):
        prev_output = states[0] if tf.nest.is_nested(states) else states
        i = tf.keras.backend.dot(inputs, self.kernel)
        
        if self.bias is not None:
            i = tf.keras.backend.bias_add(i, self.bias)

        z = tf.keras.backend.dot(prev_output, tf.linalg.set_diag(self.recurrent_kernel, np.zeros((self.units,), dtype=int)))
        iz = tf.math.add(i, z, name='add_iz')
        v = tf.math.subtract(tf.math.square(iz,name='square_iz'), iz, name='sub_v')
        output = tf.keras.backend.clip(v, -1, 1)

        new_state = [output] if tf.nest.is_nested(states) else output
        return output, new_state

class RNN_plus_models():
    def __init__(self, timestep, noInput, noOutput, batchSize, isLRS=False, isCMF=False):
        self.timestep = timestep
        self.noInput = noInput
        self.noOutput = noOutput
        self.batchSize = batchSize
        self.isLRS = isLRS
        self.isCMF = isCMF
    
    def rnn_plus_choose_models(self):
        if (self.isLRS and self.isCMF):
            return self.rnn_plus_wLRS_wCMF_model()
        elif (self.isLRS and not self.isCMF):
            return self.rnn_plus_wLRS_wtCMF_model()
        elif (not self.isLRS and self.isCMF):
            return self.rnn_plus_wtLRS_wCMF_model()
        else:
            return self.rnn_plus_wtLRS_wtCMF_model()
        
    def rnn_plus_wLRS_wCMF_model(self):
        model = tf.keras.Sequential()
        model.add(tf.keras.layers.RNN(cell=RNN_plus_v1_cell(units=self.noInput+self.noOutput), input_shape=[self.timestep, self.noInput], unroll=False, name='RNNp_layer'))
        model.add(tf.keras.layers.Dense(self.noOutput, activation='tanh', name='MLP_layer'))
        optimizer = tf.keras.optimizers.Adam(learning_rate=customLRSchedule(hyperparams['initialLearningRate'], hyperparams['learningRateDecay'], hyperparams['decayDurationFactor'], hyperparams['numTrainingSteps']), \
                                            beta_1=hyperparams['beta1'], beta_2=hyperparams['beta2'], epsilon=hyperparams['epsilon'], amsgrad=False, name="tunedAdam")
        model.compile(optimizer=optimizer, loss = 'mse', metrics=[CustomMetricError(threshold=0.0)], run_eagerly=False)
        return model

    def rnn_plus_wLRS_wtCMF_model(self):
        model = tf.keras.Sequential()
        model.add(tf.keras.layers.RNN(cell=RNN_plus_v1_cell(units=self.noInput+self.noOutput), input_shape=[self.timestep, self.noInput], unroll=False, name='RNNp_layer'))
        model.add(tf.keras.layers.Dense(self.noOutput, activation='tanh', name='MLP_layer'))
        optimizer = tf.keras.optimizers.Adam(learning_rate=customLRSchedule(hyperparams['initialLearningRate'], hyperparams['learningRateDecay'], hyperparams['decayDurationFactor'], hyperparams['numTrainingSteps']), \
                                            beta_1=hyperparams['beta1'], beta_2=hyperparams['beta2'], epsilon=hyperparams['epsilon'], amsgrad=False, name="tunedAdam")
        model.compile(optimizer=optimizer, loss = 'mse')
        return model

    def rnn_plus_wtLRS_wCMF_model(self):
        model = tf.keras.Sequential()
        model.add(tf.keras.layers.RNN(cell=RNN_plus_v1_cell(units=self.noInput+self.noOutput), input_shape=[self.timestep, self.noInput], unroll=False, name='RNNp_layer'))
        model.add(tf.keras.layers.Dense(self.noOutput, activation='tanh', name='MLP_layer'))
        optimizer = tf.keras.optimizers.Adam(learning_rate=0.001, decay=0, name="Adam_wtlrs")
        model.compile(optimizer=optimizer, loss = 'mse', metrics=[CustomMetricError(threshold=0.0)], run_eagerly=False)
        return model

    def rnn_plus_wtLRS_wtCMF_model(self):
        model = tf.keras.Sequential()
        model.add(tf.keras.layers.RNN(cell=RNN_plus_v1_cell(units=self.noInput+self.noOutput), input_shape=[self.timestep, self.noInput], unroll=False, name='RNNp_layer'))
        model.add(tf.keras.layers.Dense(self.noOutput, activation='tanh', name='MLP_layer'))
        optimizer = tf.keras.optimizers.Adam(learning_rate=0.001, decay=0, name="Adam_wtlrs")
        model.compile(optimizer=optimizer, loss = 'mse')
        return model
        
class RNN_models():
    def __init__(self, timestep, noInput, noOutput, batchSize, isLRS=False, isCMF=False):
        self.timestep = timestep
        self.noInput = noInput
        self.noOutput = noOutput
        self.batchSize = batchSize
        self.isLRS = isLRS
        self.isCMF = isCMF
        
    def rnn_choose_models(self):
        if (self.isLRS and self.isCMF):
            return self.rnn_wLRS_wCMF_model()
        elif (self.isLRS and not self.isCMF):
            return self.rnn_wLRS_wtCMF_model()
        elif (not self.isLRS and self.isCMF):
            return self.rnn_wtLRS_wCMF_model()
        else:
            return self.rnn_wtLRS_wtCMF_model()
        
    def rnn_wLRS_wCMF_model(self):
        model = tf.keras.Sequential()
        model.add(tf.keras.layers.RNN(cell=tf.keras.layers.SimpleRNNCell(units=self.noInput+self.noOutput), input_shape=[self.timestep, self.noInput], name='SimpleRNN_layer', stateful=False))
        model.add(tf.keras.layers.Dense(self.noOutput, activation='tanh', name='MLP_layer'))
        optimizer = tf.keras.optimizers.Adam(learning_rate=customLRSchedule(hyperparams['initialLearningRate'], hyperparams['learningRateDecay'], hyperparams['decayDurationFactor'], hyperparams['numTrainingSteps']), \
                                            beta_1=hyperparams['beta1'], beta_2=hyperparams['beta2'], epsilon=hyperparams['epsilon'], amsgrad=False, name="tunedAdam_rnn")
        model.compile(optimizer=optimizer, loss = 'mse', metrics=[CustomMetricError(threshold=0.0)], run_eagerly=False)
        return model

    def rnn_wLRS_wtCMF_model(self):
        model = tf.keras.Sequential()
        model.add(tf.keras.layers.RNN(cell=tf.keras.layers.SimpleRNNCell(units=self.noInput+self.noOutput), input_shape=[self.timestep, self.noInput], name='SimpleRNN_layer', stateful=False))
        model.add(tf.keras.layers.Dense(self.noOutput, activation='tanh', name='MLP_layer'))
        optimizer = tf.keras.optimizers.Adam(learning_rate=customLRSchedule(hyperparams['initialLearningRate'], hyperparams['learningRateDecay'], hyperparams['decayDurationFactor'], hyperparams['numTrainingSteps']), \
                                            beta_1=hyperparams['beta1'], beta_2=hyperparams['beta2'], epsilon=hyperparams['epsilon'], amsgrad=False, name="tunedAdam_rnn")
        model.compile(optimizer=optimizer, loss = 'mse')
        return model
    
    def rnn_wtLRS_wCMF_model(self):
        model = tf.keras.Sequential()
        model.add(tf.keras.layers.RNN(cell=tf.keras.layers.SimpleRNNCell(units=self.noInput+self.noOutput), input_shape=[self.timestep, self.noInput], name='SimpleRNN_layer', stateful=False))
        model.add(tf.keras.layers.Dense(self.noOutput, activation='tanh', name='MLP_layer'))
        optimizer = tf.keras.optimizers.Adam(learning_rate=0.001, decay=0, name="Adam_wtlrs")
        model.compile(optimizer=optimizer, loss = 'mse', metrics=[CustomMetricError(threshold=0.0)], run_eagerly=False)
        return model
    
    def rnn_wtLRS_wtCMF_model(self):
        model = tf.keras.Sequential()
        model.add(tf.keras.layers.RNN(cell=tf.keras.layers.SimpleRNNCell(units=self.noInput+self.noOutput), input_shape=[self.timestep, self.noInput], name='SimpleRNN_layer', stateful=False))
        model.add(tf.keras.layers.Dense(self.noOutput, activation='tanh', name='MLP_layer'))
        optimizer = tf.keras.optimizers.Adam(learning_rate=0.001, decay=0, name="Adam_wtlrs")
        model.compile(optimizer=optimizer, loss = 'mse')
        return model

class LSTM_models():
    def __init__(self, timestep, noInput, noOutput, batchSize, isLRS=False, isCMF=False):
        self.timestep = timestep
        self.noInput = noInput
        self.noOutput = noOutput
        self.batchSize = batchSize
        self.isLRS = isLRS
        self.isCMF = isCMF
        
    def lstm_choose_models():
        if (self.isLRS and self.isCMF):
            return self.lstm_wLRS_wCMF_model()
        elif (self.isLRS and not self.isCMF):
            return self.lstm_wLRS_wtCMF_model()
        elif (not self.isLRS and self.isCMF):
            return self.lstm_wtLRS_wCMF_model()
        else:
            return self.lstm_wtLRS_wtCMF_model()
        
    def lstm_wLRS_wCMF_model(self):
        model = tf.keras.Sequential()
        model.add(tf.keras.layers.LSTM(units=self.noInput+self.noOutput, input_shape=[self.timestep, self.noInput],
                       activation='tanh', recurrent_activation='sigmoid', unroll =False, use_bias=True,
                       recurrent_dropout=0.0, return_sequences=False))
        model.add(tf.keras.layers.Dense(self.noOutput, activation='tanh', name='MLP_layer'))
        optimizer = tf.keras.optimizers.Adam(learning_rate=customLRSchedule(hyperparams['initialLearningRate'], hyperparams['learningRateDecay'], hyperparams['decayDurationFactor'], hyperparams['numTrainingSteps']), \
                                        beta_1=hyperparams['beta1'], beta_2=hyperparams['beta2'], epsilon=hyperparams['epsilon'], amsgrad=False, name="tunedAdam_lstm")
        model.compile(optimizer=optimizer, loss = 'mse', metrics=[CustomMetricError(threshold=0.0)], run_eagerly=False)
        return model

    def lstm_wLRS_wtCMF_model(self):
        model = tf.keras.Sequential()
        model.add(tf.keras.layers.LSTM(units=self.noInput+self.noOutput, input_shape=[self.timestep, self.noInput],
                       activation='tanh', recurrent_activation='sigmoid', unroll =False, use_bias=True,
                       recurrent_dropout=0.0, return_sequences=False))
        model.add(tf.keras.layers.Dense(self.noOutput, activation='tanh', name='MLP_layer'))
        optimizer = tf.keras.optimizers.Adam(learning_rate=customLRSchedule(hyperparams['initialLearningRate'], hyperparams['learningRateDecay'], hyperparams['decayDurationFactor'], hyperparams['numTrainingSteps']), \
                                        beta_1=hyperparams['beta1'], beta_2=hyperparams['beta2'], epsilon=hyperparams['epsilon'], amsgrad=False, name="tunedAdam_lstm")
        model.compile(optimizer=optimizer, loss = 'mse')
        return model
    
    def lstm_wtLRS_wCMF_model(self):
        model = tf.keras.Sequential()
        model.add(tf.keras.layers.LSTM(units=self.noInput+self.noOutput, input_shape=[self.timestep, self.noInput],
                       activation='tanh', recurrent_activation='sigmoid', unroll =False, use_bias=True, 
                       recurrent_dropout=0.0, return_sequences=False))
        model.add(tf.keras.layers.Dense(self.noOutput, activation='tanh', name='MLP_layer'))
        optimizer = tf.keras.optimizers.Adam(learning_rate=0.001, decay=0, name="Adam_wtlrs")
        model.compile(optimizer=optimizer, loss = 'mse', metrics=[CustomMetricError(threshold=0.0)], run_eagerly=False)
        return model
    
    def lstm_wtLRS_wtCMF_model(self):
        model = tf.keras.Sequential()
        model.add(tf.keras.layers.LSTM(units=self.noInput+self.noOutput, input_shape=[self.timestep, self.noInput],
                       activation='tanh', recurrent_activation='sigmoid', unroll =False, use_bias=True,
                       recurrent_dropout=0.0, return_sequences=False))
        model.add(tf.keras.layers.Dense(self.noOutput, activation='tanh', name='MLP_layer'))
        optimizer = tf.keras.optimizers.Adam(learning_rate=0.001, decay=0, name="Adam_wtlrs")
        model.compile(optimizer=optimizer, loss = 'mse')
        return model

class models(RNN_models, RNN_plus_models, LSTM_models):
    def __init__(self,  timestep, noInput, noOutput, batchSize, modeltype='RNN_plus', isLRS=False, isCMF=False):
        super(models).__init__()
        self.modeltype = modeltype
        self.timestep = timestep
        self.noInput = noInput
        self.noOutput = noOutput
        self.batchSize = batchSize
        self.isLRS = isLRS
        self.isCMF = isCMF
    
    def chooseModels(self):
        sLRS = 'wLRS' if self.isLRS else 'wtLRS'
        sCMF = 'wCMF' if self.isCMF else 'wtCMF'
        prefix = f'{self.modeltype.lower()}_{sLRS}_{sCMF}_tf'
        display(prefix)
        if self.modeltype == 'LSTM':
            return self.lstm_choose_models()
        elif self.modeltype == 'RNN_plus':
            return self.rnn_plus_choose_models()
        else:
            return self.rnn_choose_models()

In [ ]:
result_tf = {}
model_name = 'rnn_plus_wLRS_wtCMF_tf'
result_tf[model_name]= {}
for filename in fileslist:
    filepath = os.path.join(path,filename)
    result_tf[model_name][filename] = []
    print('++', filename)
    
    logdir = f"./logs/scalars/{model_name}_{filename}"
    tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=logdir)

    if ISMOORE_DATASETS:
        timestep = int(filename.split('.')[4].split('s')[-1])      
    else:
        timestep = int(filename.split('.')[2].split('=')[-1])
        batchsize_10x = int(filename.split('.')[1].split('=')[-1])
        hyperparams['batchSize'] = 50 if batchsize_10x%50 == 0 else 20
        print("+++ Batch Size: ", hyperparams['batchSize'])

    with open(filepath, "r") as fp:
        [noIn, noOut] = [int(x) for x in fp.readline().replace('\n', '').split(',')]
        rdf = np.genfromtxt(fp, delimiter=',')
        
    print(f"Step 1: Dividing the training and testing set with ratio 1:1 ({hyperparams['testSize']*100}%).")
    df_val, df_train = train_test_split(rdf,test_size=hyperparams['testSize'])
    print("+ Training set:   ", df_train.shape, df_val.shape)

    x_train, y_train = seperateValues(df_train, noIn, noOut, isMoore=ISMOORE_DATASETS)
    x_val, y_val = seperateValues(df_val, noIn, noOut, isMoore=ISMOORE_DATASETS)    
    for i in range( x_train.shape[ 0 ] ) :
        for j in range( x_train.shape[ 1 ] ) :
            for k in range( x_train.shape[ 2 ] ) :
                x_train[ i, j, k ] = fromBit( x_train[ i, j, k ] )
    for i in range( y_train.shape[ 0 ] ) :
        for j in range( y_train.shape[ 1 ] ) :
            y_train[ i, j ] = fromBit( y_train[ i, j ] )
    for i in range( x_val.shape[ 0 ] ) :
        for j in range( x_val.shape[ 1 ] ) :
            for k in range( x_val.shape[ 2 ] ) :
                x_val[ i, j, k ] = fromBit( x_val[ i, j, k ] )
    for i in range( y_val.shape[ 0 ] ) :
        for j in range( y_val.shape[ 1 ] ) :
            y_val[ i, j ] = fromBit( y_val[ i, j ] )
    print("+ Training set:   ", x_train.shape, y_train.shape, x_train.dtype)
    print("+ Validating set: ", x_val.shape, y_val.shape, x_val.dtype)
    
    for run_count in range(1):
        model = models(modeltype="RNN_plus", timestep=timestep, noInput=noIn, noOutput=noOut, batchSize=int(hyperparams['batchSize']), isLRS=True, isCMF=False).chooseModels()
        model_history = model.fit(
                    x_train, y_train,
                    batch_size=int(hyperparams['batchSize']),
                    verbose=0, # Suppress chatty output; use Tensorboard instead
                    epochs=int(hyperparams['numTrainingSteps']/(x_train.shape[0])),
                    validation_data=(x_val, y_val),
                    shuffle=True,
                    use_multiprocessing=False,
                    callbacks=[tensorboard_callback]
                )
        y_pred = model.predict(x_val, verbose=1, batch_size=int(hyperparams['batchSize']))
        result_tf[model_name][filename].append(round(customMetricfn(y_val, y_pred), 5)*100)

# results_dir_o = "./results/merge_results_5bigdatasets.csv"
# readfile = pd.read_csv(results_dir_o, index_col='dataset')
# readfile.index.name = None

for k, v in result_tf[model_name].items():
    result_tf[model_name][k] = round(sum(result_tf[model_name][k]) / len(result_tf[model_name][k]), 2)

model_result = pd.DataFrame.from_dict(result_tf[model_name], orient='index', columns=[model_name])
display(model_result)
# pd.concat([model_result, readfile], axis=1).to_csv(results_dir_o, index=True, index_label='dataset', mode='w') 